<a href="https://colab.research.google.com/github/SY-256/anomaly_detection/blob/main/notebook/chapter4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ホテリング理論による異常検知
ホテリング理論は、解析計算に基づく理論の明快さや、統計ライブラリによる実装のしやすさから、基礎的な異常検知アルゴリズムの一つとして広く利用されている

1変数のホテリング理論による異常検知をPythonで実装
- A. データが正規分布に従うか
- B. モデルの学習
- C. 推論

## A. データが正規分布に従うか
- ヒストグラムによる可視化
- Q-Qプロットによる可視化
- 正規性検定

### ヒストグラムによる可視化
- 山が一つ
- 左右対称
- 釣鐘形の分布形状
- 裾が長すぎない（外れ値が少ない）

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/ch2_dataset_train.csv")
df_dropna = df.dropna(subset=["temp1"]) # 欠損値削除
df_normal = df_dropna[df_dropna["label"] == "normal"] # 正常データのみ抽出
x_train = df_normal["temp1"].to_numpy() # numpy.ndarrayに変換

# 変数"temp1"のヒストグラムを表示
plt.hist(x=x_train, color="#888888", bins="sturges")
plt.show()

## Q-Qプロット
- 正規性をシンプルに判断できる（データが正規分布に従うか）

1. データを昇順ソートし、分位点を求める
2. データに対して最尤推定で正規分布をフィッティングし、求めた正規分布の分位点を求める
3. 1で求めたデータの分位点を縦軸、2で求めた正規分布の分位点を横軸にとり、散布図をプロットする（この散布図をQ-Qプロットとして用いる）
4. 3で作成したQ-Qプロットがおおむね直線に沿っていれば、データが正規分布に従うとみなす

In [ ]:
# Q-Qプロット
from scipy import stats

stats.probplot(x_train, dist="norm", plot=plt)

- 多少のずれはあるものの、Q-Qプロットがおおむね直線に沿っており、データが正規分布に従っていると判断できる

## 正規性検定
- 定量的に正規性を判断できる
- サンプルサイズが小さい場合：シャピロ-ウィルク検定
- サンプルサイズが大きい場合：コルモゴロフ-スミルノフ検定

正規性検定とは、データが正規分布に従っているかを統計的に検定する方法全般を指す

In [ ]:
# 正規性検定
import numpy as np

# Shapiro-Wilk検定
stat, p = stats.shapiro(x_train)
print("Shapiro-Wilk検定の統計量=%.3f, p値=%.3f" % (stat, p))
# Kolmogorov-Smirnov検定（平均と標準偏差を`args`引数に与える必要がある）
mean = np.mean(x_train)
std = np.std(x_train, ddof=1)
stat, p = stats.kstest(x_train, stats.norm.cdf, args=(mean, std))
print("Klomogorov-Smirnov検定の統計量=%.3f, p値=%.3f" % (stat, p))

- これらの検定はデータが正規分布からサンプリングされたものであるという帰無仮説を用いている。
- p値が有意水準を下回った際に、データが正規分布に従わないことは裏付けられるが、有意水準を上回ったからといって、データが正規分布に従うことを裏付けるわけではない。
- 最終的にはヒストグラムやQ-Qプロットの当てはまりの良さなども加味して判断する。

## モデル学習
1変数のホテリング理論では、サンプルサイズNの大小に応じて以下脳ように異常度

<img src="https://latex.codecogs.com/svg.image?
a(x) = \Big(\frac{x-\hat{\mu}}{\hat{\sigma}}\Big)^2
" />

が従う確率分布を使い分けます（$\hat{\mu}$は標本平均、$\hat{\sigma}$は標本標準偏差）。

- サンプルサイズ$N$が**大きい**場合：異常度の分布は自由度1の**カイ二乗分布**に近似できる
- サンプルサイズ$N$が**小さい**場合：異常度を$\frac{N-1}{N+1}$倍した統計量は自由度$(1,N-1)$の**F分布**に従う

カイ二乗分布とF分布の使い分けにおける$N$の明確な境界は決まっていないが、1変数においてはおおむね$N\geq 100$であれば、カイ二乗分布による近似を利用したほうが良い

### サンプルサイズ$N$が大きい場合の学習の実装

サンプルサイズ$N$が大きい場合、異常度$a(x) = \Big(\frac{x-\hat{\mu}}{\hat{\sigma}}\Big)^2$が自由度1のカイ二乗分布に従うことを前提に。以下のように正規分布（正常のモデル）のパラメータ$\hat{\mu}$（コード中の`mu`）、$\hat{\sigma}$（`sigma`）および異常度の閾値$a_{th}$（`a_th`）を求める。

In [ ]:
# ホテリング理論による異常検知（サンプルサイズNが大きい場合）の実装例（学習）
import pandas as pd
import numpy as np
from scipy import stats

##### 学習データの読み込みと前処理 #####
df = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/ch2_dataset_train.csv")
# "temp1"変数に欠損があるデータを削除
df_dropna = df.dropna(subset=["temp1"])
# 正常データのみ抽出
df_normal = df_dropna[df_dropna["label"] == "normal"]
# "temp1"列のみ取り出してNumpyのndarray化し、学習データとする
x_train = df_normal["temp1"].to_numpy()

##### 学習ステップ1. 正常のモデルを作成する #####
mu = np.mean(x_train) # 標本平均μ
sigma = np.std(x_train, ddof=0) # 標本標準偏差σを算出

##### 学習ステップ2. 異常を表す指標（異常度）を定義する #####
# 上のMarkdownで式を定義するのみ

##### 学習ステップ3. 異常度に閾値を設ける #####
TARGET_FP_RATE = 0.0027 # ターゲットとする誤報率（正規分布の3σ相当=0.0027）
# 自由度1のカイ二乗分布の累積分布関数の逆関数（生存関数）から閾値を算出
a_th = stats.chi2.ppf(1-TARGET_FP_RATE, df=1)

##### 学習で求めたパラメータをすべて表示 #####
print(f"mu={mu}")
print(f"sigma={sigma}")
print(f"a_th={a_th}")

## サンプルサイズ$N$が小さい場合の学習の実装

サンプルサイズ$N$が小さい場合、異常度を$a(x) = \Big(\frac{x-\hat{\mu}}{\hat{\sigma}}\Big)^2$を$\frac{N-1}{N+1}$倍した統計量が自由度$(1, N-1)$のF分布に従うことを前提に、以下のように正規分布（正常のモデル）のパラメータ$\hat{\mu}$（コード中の`mu`）、$\hat{\sigma}$（`sigma`）および異常度の閾値$a_{th}$（`a_th`）を求める。

In [ ]:
# 途中まで同じ

##### 学習ステップ3. 異常度に閾値を設ける #####
TARGET_FP_RATE = 0.0027
sample_size = len(x_train) # サンプルサイズN
# 自由度（1, N-1）のF分布の累積密度関数の逆関数から閾値を算出
a_th = (sample_size - 1) / (sample_size + 1) \
* stats.f.ppf(1-TARGET_FP_RATE, dfn=1, dfd=sample_size-1)

##### 学習で求めたパラメータをすべて表示 #####
print(f"mu={mu}")
print(f"sigma={sigma}")
print(f"a_th={a_th}")

カイ二乗分布を使用した場合と比較して異常度の閾値がF分布を使用したほうが大きくなっており、誤報率を減らす方向（見逃し寄り）に閾値が設定されている。

求めたパラメータ`mu`（標本平均$\hat{\mu}$）、`sigma`（標本標準偏差$\hat{\sigma}$）および異常度の閾値`a_th`（$a_{th}$）は設定ファイルなどに保存しておき、推論時に利用する

## 推論

In [ ]:
# ホテリング理論による異常検知の実装例（推論）
###### 学習で求めたパラメータをここに記載 ######
MU=400.21552883243925 # 標本平均μ
SIGMA=4.958226656997426 # 標本標準偏差σ
A_TH=8.99986195674967 # 異常度のしきい値

###### 推論データの読み込みと前処理 ######
df_inference = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/ch2_dataset_inference.csv")

# "temp1"変数に欠損があるデータを削除
df_inference_dropna = df_inference.dropna(subset="temp1")
# "temp1"列のみ取り出してNumPyのndarray化し、推論データとする
x_inference = df_inference_dropna["temp1"].to_numpy()

###### 推論を実行 ######
# 異常度を算出
anomaly_scores = ((x_inference-MU)/SIGMA) ** 2
# 閾値により異常の有無を判定
pred = np.where(anomaly_scores > A_TH, "anomaly", "normal")
# 推論結果を表示
print(pred)

推論データの異常度のヒストグラムと閾値（Threshold）を重ねてプロット

In [ ]:
# 推論データの異常度のヒストグラムと閾値を重ねてプロット（正常データと異常データで色分け）
import matplotlib.pyplot as plt
import seaborn as sns

# 異常度をDataFrameに列として追加
df_inference_dropna["anomaly_score"] = anomaly_scores
# 描画用のFigureとAxesを生成
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

# 正常データのヒストグラム描画
df_inference_norm = df_inference_dropna[df_inference_dropna["label"] == "normal"]
sns.histplot(data=df_inference_norm["anomaly_score"], binwidth=1,
             color="#bbbbbb", ax=ax, stat="density", label="normal")
# 異常データのヒストグラム
df_inference_anom = df_inference_dropna[df_inference_dropna["label"] == "anomaly"]
sns.histplot(data=df_inference_anom["anomaly_score"], binwidth=1,
             color="#111111", ax=ax,
             stat="density", label="anomaly")

# 閾値の線を描画
ax.vlines(A_TH, 0, 0.3, color="black", lw=2)
ax.text(A_TH, 0.3, f"Threshold={round(A_TH, 4)}",
        verticalalignment="bottom", horizontalalignment="center")
# 凡例を表示
ax.legend()
plt.show()

検知精度は高くない

使用する変数を`temp1`から`temp2`に変更して、異常度を再度求めてみる

In [ ]:
# "temp2"変数の推論データの異常度のヒストグラムと閾値を重ねてプロット

###### 学習データの読み込みと前処理 ######
# "temp2"列のみ取り出してNumPyのndarray化し、学習データとする
x_train = df_normal["temp2"].to_numpy()

###### 学習ステップ1. 正常のモデル作成 ######
mu = np.mean(x_train) # 標本平均μ
sigma = np.std(x_train, ddof=0) # 標本標準偏差σ

###### 学習ステップ2. 異常を表す指標（異常度）を定義する ######
# 式のみ

###### 学習ステップ3. 異常度に閾値を設ける ######
TARGET_FP_RATE = 0.0027 # ターゲットとする誤報率（正規分布の3σ相当=0.0027）
sample_size = len(x_train) # サンプルサイズN
# 自由度1のカイ二乗分布の累積分布関数の逆関数から閾値算出
a_th = stats.chi2.ppf(1-TARGET_FP_RATE, df=1)

###### 推論データの前処理 ######
# "temp2"変数に欠損があるデータを削除
df_inference_dropna = df_inference.dropna(subset="temp2")
# "temp2"列のみ取り出してNumPyのndarray化し、推論データとする
x_inference = df_inference_dropna["temp2"].to_numpy()

###### 推論を実行 ######
# 異常度算出
anomaly_scores = ((x_inference-mu) / sigma) ** 2
# 閾値により異常の有無を判定
pred = np.where(anomaly_scores > a_th, "anomaly", "normal")
# 異常度をDataFrameに列として追加
df_inference_dropna["anomaly_score"] = anomaly_scores
# 描画用のFigureとAxesを生成
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

# 正常データのヒストグラム描画
df_inference_norm = df_inference_dropna[df_inference_dropna["label"] == "normal"]
sns.histplot(data=df_inference_norm["anomaly_score"], binwidth=1,
             color="#bbbbbb", ax=ax,
             stat="density", label="normal")
# 異常データのヒストグラム描画
df_inference_anom = df_inference_dropna[df_inference_dropna["label"] == "anomaly"]
sns.histplot(data=df_inference_anom["anomaly_score"], binwidth=1,
             color="#111111", ax=ax,
             stat="density", label="anomaly")
# 閾値の線を描画
ax.vlines(a_th, 0, 0.3, color="black", lw=2)
ax.text(a_th, 0.3, f"Threshold={round(a_th, 4)}",
        verticalalignment="bottom", horizontalalignment="center")
# 凡例を表示
ax.legend()
plt.show()

精度良く異常を検知できている

## 元の変数に閾値を設ける場合（異常度を使用しない）
**異常度に閾値を設ける方法と比べて、以下のようなメリットがある**
- ドメイン知識との親和性：温度や湿度など、意味が明確な変数に対して閾値を直接設定できるため、現場のエンジニアや運用担当者にも理解しやすい
- 上下各方向の制御が可能：上限・下限に別々の閾値を設定できるため、異常の方向性（高すぎる・低すぎる）に応じて個別に感度を制御できる

一方でこの方法では、**多変数への拡張が困難**であることや、複数の閾値を持つことから**解釈や運用の複雑性が上がる**ことに注意が必要

サンプルサイズ$N$が大きい（正規分布に基づき閾値を算出する）場合の実装

In [ ]:
# ホテリング理論による異常検知
###### 学習ステップ1. 正常のモデルを作成する ######
mu = np.mean(x_train) # 標本平均μ
sigma = np.std(x_train, ddof=1) # 標本標準偏差σ

###### 学習ステップ2. 異常を表す指標（異常度）を定義する ######
# 異常度は使用しない

###### 学習ステップ3. 元の変数に閾値を設ける ######
TARGET_FP_RATE = 0.0027 # ターゲットとする誤報率（正規分布の3σ相当=0.0027）
# 正規分布の累積分布関数の逆関数から下限閾値算出（誤報率は2で割る）
th_lower = stats.norm.ppf(TARGET_FP_RATE/2, loc=mu, scale=sigma)
# 正規分布の累積分布関数の逆関数から上側閾値算出（誤報率は2で割る）
th_upper = stats.norm.ppf(1 - TARGET_FP_RATE/2, loc=mu, scale=sigma)

# 学習で求めたパラメータをすべて表示
print(f"mu={mu}")
print(f"sigma={sigma}")
print(f"th_lower={th_lower}")
print(f"th_upper={th_upper}")

サンプルサイズ$N$が小さい（スチューデントのt分布に基づき閾値を算出する）場合、以下のように実装

In [ ]:
###### 学習ステップ3. 異常度に閾値を設ける ######
TARGET_FP_RATE = 0.0027
sample_size = len(x_train)
# 自由度N-1のt-分布の累積分布関数の逆関数から下側閾値算出（誤報率は2で割る）
th_lower = mu + sigma * np.sqrt((sample_size+1) / (sample_size-1)) \
* stats.t.ppf(TARGET_FP_RATE/2, df=sample_size-1)
# 自由度N-1の累積分布関数の逆関数から上側閾値算出（誤報率は2で割る）
th_upper = mu + sigma * np.sqrt((sample_size+1) / (sample_size-1)) \
* stats.t.ppf(1 - TARGET_FP_RATE/2, df=sample_size-1)

# 学習で求めたパラメータをすべて表示
print(f"mu={mu}")
print(f"sigma={sigma}")
print(f"th_lower={th_lower}")
print(f"th_upper={th_upper}")

## 推論
異常度に閾値設ける場合との違いは、上下両方向に対してそれぞれ閾値を設定する